In [20]:
import numpy as np
import pandas as pd

In [80]:
def calculate_performance_loss(baseline_acc, current_acc):
    """Calculate normalized performance loss."""
    #print('check: baseline_acc: ', baseline_acc, 'current_acc: ', current_acc)
    return (current_acc - baseline_acc) / baseline_acc if baseline_acc != 0 else 0

def calculate_differential_losses(pruning_results, baseline):
    """
    Calculate the differential of performance losses across pruning ratios.
    Arguments:
    pruning_results -- list of tuples, where each tuple contains (pruning_ratio, obj_acc, rel_acc, triplet_acc)
    baseline -- tuple of (baseline_obj_acc, baseline_rel_acc, baseline_triplet_acc)
    """
    # Sorting by pruning ratios for accurate differential calculation
    sorted_results = sorted(pruning_results, key=lambda x: x[0])
    ratios = [x[0] for x in sorted_results]
    losses = [[calculate_performance_loss(baseline[i], x[i + 1]) for i in range(3)] for x in sorted_results]
    
    # Calculating differentials of losses
    diffs = -np.diff(np.array(losses), axis=0)
    diffs = np.vstack([diffs, diffs[-1]])  # Duplicate last differential as an approximation for the last point
    return ratios, losses, diffs

def calculate_model_score(losses, diffs, weights=(1, 1, 5), mu=5, pruning_ratio=0, detailed=False):
    """Calculate scores incorporating the absolute values of losses and their differentials, adjusted by lambda."""
    adjusted_losses = losses
    adjusted_diffs = diffs
    #print(f'pruning ratio: {pruning_ratio}, adjusted losses: {adjusted_losses}, adjusted diffs: {adjusted_diffs}')
    weighted_diff_loss = sum(weights[i] * (adjusted_diffs[i] + adjusted_losses[i]) for i in range(3))
    score = weighted_diff_loss + mu * pruning_ratio
    
    if detailed:
        return score, weighted_diff_loss
    return score

In [5]:

base_dir = '/home/song/Desktop/song/lightweight_3DSSG/visualization/pruning_infer_data/'
# CSV 파일 경로
file_path1 = base_dir + 'sgfn_baseline_st_results.csv'
file_path2 = base_dir +'attn_sgfn_baseline_st_results.csv'
file_path3 = base_dir +'sgpn_baseline_st_results.csv'
file_paht4 = base_dir + 'VLSAT_param65_Structured_pruninig_results.csv'
# CSV 파일 로드
df1 = pd.read_csv(file_path1).sort_values(by='Pruning Ratio')
df2 = pd.read_csv(file_path2).sort_values(by='Pruning Ratio')
df3 = pd.read_csv(file_path3).sort_values(by='Pruning Ratio')
df4 = pd.read_csv(file_paht4).sort_values(by='Pruning Ratio')


# 필요한 데이터를 추출하여 리스트로 변환
sgfn_pruning_results = [
    (row['Pruning Ratio'], row['3d obj Acc@1'], row['3d rel Acc@1'], row['3d triplet Acc@50'])
    for index, row in df1.iterrows()
]
sgfn_baseline = (
    df1['3d obj Acc@1'].iloc[0],
    df1['3d rel Acc@1'].iloc[0],
    df1['3d triplet Acc@50'].iloc[0]
)
attn_sgfn_pruning_results = [
    (row['Pruning Ratio'], row['3d obj Acc@1'], row['3d rel Acc@1'], row['3d triplet Acc@50'])
    for index, row in df2.iterrows()
]
attn_sgfn_baseline = (
    df2['3d obj Acc@1'].iloc[0],
    df2['3d rel Acc@1'].iloc[0],
    df2['3d triplet Acc@50'].iloc[0]
)

sgpn_pruning_results = [
    (row['Pruning Ratio'], row['3d obj Acc@1'], row['3d rel Acc@1'], row['3d triplet Acc@50'])
    for index, row in df3.iterrows()
]
sgpn_baseline = (
    df3['3d obj Acc@1'].iloc[0],
    df3['3d rel Acc@1'].iloc[0],
    df3['3d triplet Acc@50'].iloc[0]
)

vlsat_pruning_results = [
    (row['Pruning Ratio'], row['3d obj Acc@1'], row['3d rel Acc@1'], row['3d triplet Acc@50'])
    for index, row in df4.iterrows()
]
vlsat_baseline = (
    df4['3d obj Acc@1'].iloc[0],
    df4['3d rel Acc@1'].iloc[0],
    df4['3d triplet Acc@50'].iloc[0]
)
print("SGFN Pruning Results:", sgfn_pruning_results)
print("SGFN Baseline:", sgfn_baseline)
print("Attention SGFN Pruning Results:", attn_sgfn_pruning_results)
print("Attention SGFN Baseline:", attn_sgfn_baseline)
print("SGPN Pruning Results:", sgpn_pruning_results)
print("SGPN Baseline:", sgpn_baseline)
print("VLSAT Pruning Results:", vlsat_pruning_results)
print("VLSAT Baseline:", vlsat_baseline)


SGFN Pruning Results: [(0.0, 52.66, 89.97, 88.53), (0.05, 52.48, 90.05, 88.43), (0.1, 52.48, 89.87, 88.25), (0.15, 52.67, 89.36, 88.11), (0.2, 51.99, 88.79, 87.92), (0.25, 51.13, 87.03, 87.26), (0.3, 50.52, 85.58, 86.6), (0.35, 47.36, 85.73, 84.57), (0.4, 41.54, 81.21, 80.28), (0.45, 36.97, 73.04, 77.1), (0.5, 32.31, 24.39, 74.83), (0.55, 25.02, 19.25, 73.46), (0.6, 16.86, 17.5, 73.12), (0.65, 14.42, 15.7, 72.69), (0.7, 13.76, 12.29, 72.44), (0.75, 14.94, 6.87, 72.36), (0.8, 10.07, 5.58, 72.29), (0.85, 4.07, 5.57, 72.34), (0.9, 4.07, 5.57, 72.34), (0.95, 4.07, 5.57, 72.34)]
SGFN Baseline: (52.66, 89.97, 88.53)
Attention SGFN Pruning Results: [(0.0, 54.77, 88.45, 88.7), (0.05, 54.63, 88.02, 89.14), (0.1, 54.84, 86.54, 89.06), (0.15, 53.93, 84.01, 88.82), (0.2, 53.04, 81.12, 88.34), (0.25, 50.79, 76.34, 87.59), (0.3, 49.02, 71.05, 86.64), (0.35, 47.61, 65.29, 86.22), (0.4, 44.13, 60.21, 84.93), (0.45, 42.47, 52.87, 83.67), (0.5, 41.45, 54.81, 81.9), (0.55, 37.37, 52.88, 80.12), (0.6, 32.

In [6]:
models_data = {
    "SGPN": {
        "baseline": sgpn_baseline,
        "pruning_results": sgpn_pruning_results
    },
    "SGFN": {
        "baseline": sgfn_baseline,
        "pruning_results": sgfn_pruning_results
    },
    "Attn + SGFN": {
        "baseline": attn_sgfn_baseline,
        "pruning_results": attn_sgfn_pruning_results
    },
    "VLSAT": {
        "baseline": vlsat_baseline,
        "pruning_results": vlsat_pruning_results
    }
    
}

In [7]:
def display_scores(models_data, weights=(0, 0, 5), mu=0.5):
    results = {}
    for model_name, data in models_data.items():
        baseline = data["baseline"]
        pruning_results = data["pruning_results"]
        if not pruning_results or len(pruning_results) < 2:
            print(f"Not enough data to calculate scores for {model_name}.")
            continue
        
        ratios, losses, diffs = calculate_differential_losses(pruning_results, baseline)
        
        # Store detailed score components for each pruning ratio
        detailed_scores = []
        scores = []
        for idx in range(len(ratios)):
            score, components = calculate_model_score(losses[idx], diffs[idx], weights, mu, ratios[idx], detailed=True)
            scores.append(score)
            detailed_scores.append({
                "ratio": ratios[idx],
                "weighted_diff_loss": components,
                "weighted_ratio": mu * ratios[idx],
                "total_score": score
            })
        
        max_score_index = np.argmax(scores)
        best_ratio = ratios[max_score_index]
        best_score = scores[max_score_index]

        results[model_name] = {
            "best_ratio": best_ratio,
            "best_score": best_score,
            "all_scores": scores,
            "detailed_scores": detailed_scores
        }

        print(f"Scores for {model_name}:")
        for entry in detailed_scores:
            print(f"  Pruning Ratio {entry['ratio']}, Weighted Diff-Loss: {entry['weighted_diff_loss']}, "
                  f"Weighted Ratio: {entry['weighted_ratio']:.3f}, Total Score: {entry['total_score']:.3f}")
        print(f"Best Pruning Ratio: {best_ratio}, Highest Score: {best_score:.3f}\n")

    return results

In [104]:
final_results = display_scores(models_data)
#final_results

Scores for SGPN:
  Pruning Ratio 0.0, Weighted Diff-Loss: 0.3723979298447383, Weighted Ratio: 0.000, Total Score: 0.372
  Pruning Ratio 0.05, Weighted Diff-Loss: -0.22277170787809059, Weighted Ratio: 0.025, Total Score: -0.198
  Pruning Ratio 0.1, Weighted Diff-Loss: -0.38959171937895376, Weighted Ratio: 0.050, Total Score: -0.340
  Pruning Ratio 0.15, Weighted Diff-Loss: -0.6799884991374354, Weighted Ratio: 0.075, Total Score: -0.605
  Pruning Ratio 0.2, Weighted Diff-Loss: -0.6448533640023001, Weighted Ratio: 0.100, Total Score: -0.545
  Pruning Ratio 0.25, Weighted Diff-Loss: -0.5637147786083956, Weighted Ratio: 0.125, Total Score: -0.439
  Pruning Ratio 0.3, Weighted Diff-Loss: -0.6723979298447373, Weighted Ratio: 0.150, Total Score: -0.522
  Pruning Ratio 0.35, Weighted Diff-Loss: -0.7207015526164471, Weighted Ratio: 0.175, Total Score: -0.546
  Pruning Ratio 0.4, Weighted Diff-Loss: -0.5920069005175389, Weighted Ratio: 0.200, Total Score: -0.392
  Pruning Ratio 0.45, Weighted Dif

In [1]:
import pandas as pd

# SGFN 구조화된 및 비구조화된 프루닝 결과
vlsat_structured_data = {
    "Pruning Ratio": [0.7, 0.55, 0.3, 0.25, 0.2, 0, 0.35, 0.45, 0.05, 0.65, 0.4, 0.1, 0.6, 0.15, 0.5, 0.75],
    "3d obj Acc@1": [41.581, 53.214, 55.258, 55.385, 55.385, 55.406, 55.216, 55.216, 55.427, 45.922, 55.258, 55.490, 49.905, 55.385, 54.984, 37.450],
    "3d rel Acc@1": [38.845, 38.845, 65.470, 66.546, 75.195, 91.175, 66.152, 56.646, 87.311, 38.845, 68.807, 84.512, 38.845, 80.078, 49.831, 5.830],
    "3d triplet Acc@50": [79.280, 85.169, 88.605, 88.712, 88.870, 89.701, 88.387, 87.965, 89.329, 81.791, 88.070, 89.187, 84.094, 89.068, 87.227, 77.621]
}

unstructured_data = {
    "Pruning Ratio": [0.8, 0.65, 0.85, 0.75, 0.45, 0.5, 0.35, 0.3, 0.25, 0.6, 0.05, 0.7, 0.1, 0.4, 0.9, 0.15, 0.95, 0.2, 0.55],
    "3d obj Acc@1": [38.609, 49.631, 32.455, 43.267, 51.949, 51.823, 52.055, 52.308, 52.624, 50.896, 52.476, 46.575, 52.476, 51.970, 23.878, 52.497, 4.067, 52.518, 51.149],
    "3d rel Acc@1": [85.145, 89.071, 79.773, 87.398, 89.807, 89.681, 89.879, 89.998, 89.986, 89.267, 89.956, 88.337, 89.946, 89.874, 53.827, 89.973, 13.326, 89.906, 89.525],
    "3d triplet Acc@50": [82.103, 87.256, 77.862, 84.398, 88.417, 88.208, 88.528, 88.506, 88.491, 87.631, 88.508, 85.920, 88.516, 88.508, 73.616, 88.508, 72.639, 88.513, 87.928]
}
# DataFrame 생성 및 정렬
df_structured = pd.DataFrame(vlsat_structured_data).sort_values('Pruning Ratio').round(2)
#df_unstructured = pd.DataFrame(unstructured_data).sort_values('Pruning Ratio').round(2)

# CSV 파일로 저장
df_structured.to_csv('VLSAT_param65_Structured_pruninig_results.csv', index=False)
#df_unstructured.to_csv('sgfn_baseline_unst_results.csv', index=False)

print("CSV 파일이 성공적으로 저장되었습니다.")

CSV 파일이 성공적으로 저장되었습니다.
